In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing required libraries

import pandas as pd
import numpy as np
import shap
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import tree

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# reading the data
df = pd.read_csv('/kaggle/input/big-mart-data/bigmart_data.csv')

In [ ]:
# imputing missing values in Item_Weight by median and Outlet_Size with mode
df['Item_Weight'].fillna(df['Item_Weight'].median(), inplace=True)
df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0],inplace=True)

In [ ]:
# creating a broad category of type of Items
df['Item_Type_Combined'] = df['Item_Identifier'].apply(lambda df: df[0:2])
df['Item_Type_Combined'] = df['Item_Type_Combined'].map({'FD':'Food','NC':'Non-Consumable','DR':'Drinks'})

print(df['Item_Type_Combined'].value_counts())

# operating years of the store
df['Outlet_Years'] = 2013 - df['Outlet_Establishment_Year']

# modifying categories of Item_Fat_Content
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                        'reg':'Regular','low fat':'Low Fat'})
print(df['Item_Fat_Content'].value_counts())

In [ ]:
# Data preprocessing
# label encoding the ordinal variables

le = LabelEncoder()
df['Outlet'] = le.fit_transform(df['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])
    
# one hot encoding the remaining categorical variables
df = pd.get_dummies(df,columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Item_Type_Combined','Outlet'])

In [ ]:
# dropping the ID variables and variables that have been used to extract new variables
df.drop(['Item_Type','Outlet_Establishment_Year','Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

# separating the dependent and independent variables
X = df.drop('Item_Outlet_Sales',1)
y = df["Item_Outlet_Sales"]

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25,random_state=42)

In [ ]:
shap.initjs()

In [ ]:
xgb_model = XGBRegressor(n_estimators=1000,max_depth=10,learning_rate=0.001,random_state=0)
xgb_model.fit(X_train,y_train)

In [ ]:
y_predict = xgb_model.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_predict)**(0.5)

In [ ]:
explainer = shap.TreeExplainer(xgb_model)

In [ ]:
shap_values = explainer.shap_values(X_train)

In [ ]:
i = 4776

In [ ]:
shap.force_plot(explainer.expected_value,shap_values[i],features=X_train.loc[4776],feature_names=X_train.columns)

In [ ]:
shap.summary_plot(shap_values, features=X_train, feature_names=X_train.columns)